In [18]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

# Load the dataset
file_path = r'C:\Users\angin\OneDrive\Desktop\data analytics\Project_2\Airbnb_Open_Data.csv'  # Update with your file path
df = pd.read_csv(file_path)

# 1. Delete all duplicates
df.drop_duplicates(inplace=True)

# 2. Delete all additional spaces and enters between words in text columns
df = df.applymap(lambda x: " ".join(str(x).split()) if isinstance(x, str) else x)

# 3. Change uppercase words that are the titles in the columns to lowercase
# 4. Replace spaces in column titles with underscores
df.columns = df.columns.str.lower().str.replace(' ', '_')

# 5. Remove $ character from 'price' and 'service_fee' columns and convert to float
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)
df['service_fee'] = df['service_fee'].replace('[\$,]', '', regex=True).astype(float)

# 6. Handle null cells
# Add 0 for null cells in the service_fee column
df['service_fee'].fillna(0, inplace=True)

# Calculate the median for the price column and fill null values
price_median = df['price'].median()
df['price'].fillna(price_median, inplace=True)

# Fill null values in neighbourhood_group based on neighbourhood
df['neighbourhood_group'] = df.apply(
    lambda row: df[df['neighbourhood'] == row['neighbourhood']]['neighbourhood_group'].mode()[0]
    if pd.isnull(row['neighbourhood_group']) and not pd.isnull(row['neighbourhood']) else row['neighbourhood_group'],
    axis=1
)

# Predict null values in neighbourhood based on lat and long if there are null values
if df['neighbourhood'].isnull().any():
    train_data = df.dropna(subset=['neighbourhood', 'lat', 'long'])
    if not train_data.empty:
        knn = KNeighborsClassifier(n_neighbors=3)
        knn.fit(train_data[['lat', 'long']], train_data['neighbourhood'])
        # Predict neighbourhood for rows with missing values
        null_neighbourhood = df['neighbourhood'].isnull()
        df.loc[null_neighbourhood, 'neighbourhood'] = knn.predict(df.loc[null_neighbourhood, ['lat', 'long']])

# Replace null values in text columns with "Unspecified" and in numeric columns with NaN
df.fillna(value={col: 'Unspecified' if df[col].dtype == 'object' else np.nan for col in df.columns}, inplace=True)

# Replace null cells in last_review column with NaT
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

# 7. Delete specified columns
columns_to_remove = ['license', 'country_code', 'country']
df.drop(columns=[col for col in columns_to_remove if col in df.columns], inplace=True)

# 8. Provide the types of each column for importing the data to pgAdmin
print(df.dtypes)

# 9. Standardize text data and fix specific misspellings
df = df.applymap(lambda x: x.title().strip() if isinstance(x, str) else x)
df['neighbourhood_group'] = df['neighbourhood_group'].replace({'Brookln': 'Brooklyn', 'Manhatan': 'Manhattan'})

# 10. For "availability_365" column make the numbers of absolute type and limit the maximum value to "366"
df['availability_365'] = df['availability_365'].abs().clip(upper=366)

# Save the cleaned data to a new CSV file
output_file_path = r'C:\Users\angin\OneDrive\Desktop\data analytics\Project_2\Airbnb_Open_Data_Zzveci.csv'
df.to_csv(output_file_path, index=False)

print("Cleaned data has been saved to:", output_file_path)


C:\Users\angin\AppData\Local\Temp\ipykernel_29540\1022721962.py:7: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\angin\AppData\Local\Temp\ipykernel_29540\1022721962.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: " ".join(str(x).split()) if isinstance(x, str) else x)


id                                         int64
name                                      object
host_id                                    int64
host_identity_verified                    object
host_name                                 object
neighbourhood_group                       object
neighbourhood                             object
lat                                      float64
long                                     float64
instant_bookable                          object
cancellation_policy                       object
room_type                                 object
construction_year                        float64
price                                    float64
service_fee                              float64
minimum_nights                           float64
number_of_reviews                        float64
last_review                       datetime64[ns]
reviews_per_month                        float64
review_rate_number                       float64
calculated_host_list

C:\Users\angin\AppData\Local\Temp\ipykernel_29540\1022721962.py:62: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.title().strip() if isinstance(x, str) else x)


Cleaned data has been saved to: C:\Users\angin\OneDrive\Desktop\data analytics\Project_2\Airbnb_Open_Data_Zzveci.csv


In [23]:
!pip install geopy

from geopy.geocoders import Nominatim

# Initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises")

# Latitude and Longitude input
latitude = 40.7158
longitude = -73.95803

# Get location
location = geolocator.reverse((latitude, longitude), exactly_one=True)

# Print location details
print(location.address)


604, Driggs Avenue, Williamsburg, Brooklyn, Kings County, City of New York, New York, 11211, United States


In [24]:
# Latitude and Longitude input
latitude = 40.75348
longitude = -73.97065

# Get location
location = geolocator.reverse((latitude, longitude), exactly_one=True)

# Print location details
print(location.address)

Citi bike - East 47th Street & 2nd Avenue, East 47th Street, Beekman, Manhattan Community Board 6, Manhattan, New York County, City of New York, New York, 10017, United States
